# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
from secrets import IEX_CLOUD_API_TOKEN

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
#stks = pd.read_excel('trades.xlsx')
#stks

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = 'AAPL'
api = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2204002335922,
 'week52high': 148.72,
 'week52low': 53.72,
 'week52highSplitAdjustOnly': 145.51,
 'week52lowSplitAdjustOnly': 54.07,
 'week52change': 0.6760414060993563,
 'sharesOutstanding': 16790328473,
 'float': 0,
 'avg10Volume': 92616983,
 'avg30Volume': 102390212,
 'day200MovingAvg': 122.81,
 'day50MovingAvg': 136.09,
 'employees': 150721,
 'ttmEPS': 3.76,
 'ttmDividendRate': 0.8195243832799343,
 'dividendYield': 0.006325045561975243,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-28',
 'nextEarningsDate': '2021-01-22',
 'peRatio': 35.21037817058446,
 'beta': 1.188341766967959,
 'maxChangePercent': 50.15696368380838,
 'year5ChangePercent': 4.832146093451566,
 'year2ChangePercent': 2.12431774139671,
 'year1ChangePercent': 0.6894363030829005,
 'ytdChangePercent': -0.020282110928723115,
 'month6ChangePercent': 0.04961947693593047,
 'month3ChangePercent': 0.11332266264078207,
 'month1ChangePercent': -0.05031162537831586,
 'day30Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.6894363030829005

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote,news,chart&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        df = df.append(pd.Series([symbol,data[symbol]['price'],data[symbol]['stats']['year1ChangePercent'],'N/A'],index=my_columns),ignore_index=True)
        
df    

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,128.30,0.519598,N/A
1,AAL,21.07,-0.33424,N/A
2,AAP,172.50,0.19725,N/A
3,AAPL,127.00,0.696872,N/A
4,ABBV,109.11,0.16985,N/A
...,...,...,...,...
500,YUM,110.04,0.037254,N/A
501,ZBH,167.19,0.00346,N/A
502,ZBRA,501.34,1.076954,N/A
503,ZION,53.61,0.180346,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
df.reset_index(inplace = True)
df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,37.30,2.214487,N/A
1,179,FCX,38.33,2.163477,N/A
2,387,PYPL,283.16,1.444993,N/A
3,467,VIAC,68.38,1.321093,N/A
4,24,ALGN,595.37,1.306728,N/A
5,266,KLAC,323.39,1.130887,N/A
6,410,SIVB,551.60,1.117531,N/A
7,275,LB,51.84,1.110025,N/A
8,502,ZBRA,501.34,1.076954,N/A
9,6,ABMD,325.91,1.0397,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
position_size = float(input('Investment')) / len(df.index)
df['Number of Shares to Buy'] = df.apply(lambda x: math.floor(position_size / x['Price']),axis=1)
df

Investment10000000


<ipython-input-10-5323145526f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Number of Shares to Buy'] = df.apply(lambda x: math.floor(position_size / x['Price']),axis=1)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,37.30,2.214487,5361
1,179,FCX,38.33,2.163477,5217
2,387,PYPL,283.16,1.444993,706
3,467,VIAC,68.38,1.321093,2924
4,24,ALGN,595.37,1.306728,335
5,266,KLAC,323.39,1.130887,618
6,410,SIVB,551.60,1.117531,362
7,275,LB,51.84,1.110025,3858
8,502,ZBRA,501.34,1.076954,398
9,6,ABMD,325.91,1.0397,613


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [36]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   0,
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   0,
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   0,
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                    0,
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   0,
                                                   0
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.80,0,0.504979,0,0.304181,0,0.152658,0,-0.001061,0,0
1,AAL,20.61,0,-0.338931,0,0.548666,0,0.506755,0,0.188969,0,0
2,AAP,175.60,0,0.195889,0,0.068701,0,0.157333,0,-0.004318,0,0
3,AAPL,131.00,0,0.702392,0,0.049874,0,0.109055,0,-0.050487,0,0
4,ABBV,110.51,0,0.167752,0,0.140672,0,0.054465,0,-0.05717,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.79,0,0.036382,0,0.112547,0,0.011056,0,-0.020919,0,0
501,ZBH,167.28,0,0.003453,0,0.157668,0,0.078278,0,-0.040526,0,0
502,ZBRA,512.18,0,1.102953,0,0.789372,0,0.384118,0,0.218953,0,0
503,ZION,54.45,0,0.186683,0,0.673845,0,0.364532,0,0.102582,0,0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [ ]:
# Drop non values in the dataframe (in place) so score function can operate
hqm_dataframe.dropna(inplace=True)

In [43]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100


#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.80,0,0.504979,0.876248,0.304181,0.640719,0.152658,0.628743,-0.001061,0.37525,0
1,AAL,20.61,0,-0.338931,0.025948,0.548666,0.828343,0.506755,0.966068,0.188969,0.97006,0
2,AAP,175.60,0,0.195889,0.630739,0.068701,0.261477,0.157333,0.636727,-0.004318,0.343313,0
3,AAPL,131.00,0,0.702392,0.942116,0.049874,0.237525,0.109055,0.52495,-0.050487,0.129741,0
4,ABBV,110.51,0,0.167752,0.572854,0.140672,0.41517,0.054465,0.391218,-0.05717,0.103792,0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.79,0,0.036382,0.40519,0.112547,0.361277,0.011056,0.275449,-0.020919,0.251497,0
501,ZBH,167.28,0,0.003453,0.339321,0.157668,0.433134,0.078278,0.46507,-0.040526,0.169661,0
502,ZBRA,512.18,0,1.102953,0.986028,0.789372,0.936128,0.384118,0.908184,0.218953,0.982036,0
503,ZION,54.45,0,0.186683,0.616766,0.673845,0.892216,0.364532,0.896208,0.102582,0.840319,0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: